In [114]:
import pandas as pd
import plotly.express as px

# Graphs:

Hay que usar el playerId para unir los datos de los jugadores con lo de wyscout


### Player info
- nombre (dropdown que se puede buscar)
- equipo (dropdown)
- edad
- peso
- posición (la de wyscout o de *clusters idealmente)
- *costo

### Player stats

(se necesita un .csv con todas estas estadísticas aparte de )

- xT
- VAEP
- agregados de todo
- *minutos jugados

In [115]:
df_eng = pd.read_csv('./../../data/wyscout_tabular/england.csv')
df_ger = pd.read_csv('./../../data/wyscout_tabular/germany.csv')
df_ita = pd.read_csv('./../../data/wyscout_tabular/italy.csv')
df_fra = pd.read_csv('./../../data/wyscout_tabular/france.csv')
df_spa = pd.read_csv('./../../data/wyscout_tabular/spain.csv')
#df_wc = pd.read_csv('./../../data/wyscout_tabular/events_World_Cup.csv')

dfs = [df_eng, df_ger, df_ita, df_fra, df_spa]

df_events = pd.concat(dfs)

In [116]:
df_p_eng = pd.read_csv('./../../data/datos_scrappeados/players_generalEngland-Football.csv')
df_p_ger = pd.read_csv('./../../data/datos_scrappeados/players_generalGermany-Football.csv')
df_p_ita = pd.read_csv('./../../data/datos_scrappeados/players_generalItaly-Football.csv')
df_p_fra = pd.read_csv('./../../data/datos_scrappeados/players_generalFrance-Football.csv')
df_p_spa = pd.read_csv('./../../data/datos_scrappeados/players_generalSpain-Football.csv')

In [117]:
player_stats = [df_p_eng, df_p_ger, df_p_ita, df_p_fra, df_p_spa]
player_stats = pd.concat(player_stats)

In [118]:
df_players = pd.read_json('./../../data/wyscout/players.json', encoding='utf8')
df_teams = pd.read_json('./../../data/wyscout/Teams/teams.json', encoding='utf8')
df_comps = pd.read_json('./../../data/wyscout/Competitions/competitions.json', encoding='utf8')

df_players[['name', 'id', 'alpha3code', 'alpha2code']] = pd.json_normalize(df_players['passportArea'])
df_players['role'] = pd.json_normalize(df_players['role'])['name']
df_players[['country', 'cou_code']] = pd.json_normalize(df_players['birthArea'])[['name', 'alpha2code']]

df_players = df_players.drop(columns=['passportArea', 'birthArea'])

In [119]:
#fix encoding
df_teams['officialName'] = df_teams['officialName'].apply(lambda x: x.encode().decode('unicode_escape'))
df_teams['name'] = df_teams['name'].apply(lambda x: x.encode().decode('unicode_escape'))

df_players['firstName'] = df_players['firstName'].apply(lambda x: x.encode().decode('unicode_escape'))
df_players['lastName'] = df_players['lastName'].apply(lambda x: x.encode().decode('unicode_escape'))
df_players['middleName'] = df_players['middleName'].apply(lambda x: x.encode().decode('unicode_escape'))

In [120]:
df_players = df_players[['firstName', 'middleName', 'lastName', 'currentTeamId', 'weight', 
                         'height', 'foot', 'country', 'cou_code', 'role']]

#some players dont have club assigned. It's because of players from the wold cup that don't have a team on the top 5 leagues
df_all_info = df_players.merge(df_teams[['city', 'name', 'officialName', 'wyId']], left_on='currentTeamId', right_on='wyId', how='left')

In [121]:
text = "O.G.C. Nice C\u00f4te d'Azur"
text = text.encode().decode('unicode_escape')
print(text)

O.G.C. Nice CÃ´te d'Azur


### Unir datos de jugadores

df_all_info:
- 3603 jugadores
- pero 1164 no tienen equipo asignado (

In [122]:
def get_last_name_1(text):
    text_list = text.split(" ")
    
    return text_list[0]

In [123]:
df_all_info = df_all_info[df_all_info['officialName'].isna() == False]
df_all_info.shape

(2439, 14)

In [124]:
df_all_info['last_name_1'] = df_all_info['lastName'].apply(lambda x: get_last_name_1(x))
df_all_info['first_name_1'] = df_all_info['firstName'].apply(lambda x: get_last_name_1(x))

In [125]:
df_all_info['fullName'] = df_all_info['first_name_1'] + ' ' + df_all_info['last_name_1']

In [126]:
#600 names don't match

df_all_info.shape

(2439, 17)

In [127]:
#1162 don't match

player_stats.shape

(3164, 30)

In [128]:
df_all_info[df_all_info.officialName.str.contains("Tottenham") == True]

,firstName,middleName,lastName,currentTeamId,weight,height,foot,country,cou_code,role,city,name,officialName,wyId,last_name_1,first_name_1,fullName
6,Toby,,Alderweireld,1624,91,187,right,Belgium,BE,Defender,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Alderweireld,Toby,Toby Alderweireld
7,Jan,,Vertonghen,1624,88,189,left,Belgium,BE,Defender,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Vertonghen,Jan,Jan Vertonghen
9,Christian,,Dannemann Eriksen,1624,76,180,right,Denmark,DK,Midfielder,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Dannemann,Christian,Christian Dannemann
718,Danny,,Rose,1624,76,174,left,England,,Defender,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Rose,Danny,Danny Rose
745,Harry,,Kane,1624,86,188,right,England,,Forward,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Kane,Harry,Harry Kane
750,Kieran,,Trippier,1624,72,178,right,England,,Defender,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Trippier,Kieran,Kieran Trippier
846,Moussa Sidi Yaya,,Dembélé,1624,88,185,left,Belgium,BE,Midfielder,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Dembélé,Moussa,Moussa Dembélé
936,Eric,,Dier,1624,90,188,right,England,,Midfielder,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Dier,Eric,Eric Dier
939,Bamidele,,Alli,1624,80,188,right,England,,Midfielder,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Alli,Bamidele,Bamidele Alli
1061,Heung-Min,,Son,1624,77,183,right,Korea Republic,KR,Forward,London,Tottenham Hotspur,Tottenham Hotspur FC,1624.0,Son,Heung-Min,Heung-Min Son


In [129]:
df_all_info[df_all_info['lastName'].str.contains("Origi") == True]

,firstName,middleName,lastName,currentTeamId,weight,height,foot,country,cou_code,role,city,name,officialName,wyId,last_name_1,first_name_1,fullName
288,Divock,,Okoth Origi,2451,75,185,right,Belgium,BE,Forward,Wolfsburg,Wolfsburg,VfL Wolfsburg,2451.0,Okoth,Divock,Divock Okoth


In [130]:
player_stats[player_stats['player'].str.contains("Origi") == True]

,Unnamed: 0,nationality,position,age,games,games_starts,minutes,minutes_90s,goals,assists,...,xg,npxg,xg_assist,npxg_xg_assist,xg_per90,xg_assist_per90,xg_xg_assist_per90,npxg_per90,npxg_xg_assist_per90,player
56,56,be BEL,FW,22.0,1,0,10,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Divock Origi
127,127,be BEL,FW,22.0,31,22,"1,967",21.9,6.0,2.0,...,7.2,7.2,1.1,8.3,0.33,0.05,0.38,0.33,0.38,Divock Origi


In [131]:
player_stats['len'] = player_stats.player.str.len()
#player_stats.sort_values(by='len', ascending=False).head(20)

# Razones por que lso nombres no cuadran:
- El first name en player_stats es como una abreviación o un apodo (Ignacio está como Nacho)
- De plano no están
- En playerstats a veces se toma su segundo apellido en vez del primero
- Nombres escritos de forma diferente
- Están de forma diferente
- Solo sale el primer nombre (Roberto Firmino)

In [137]:
df_left = df_all_info.merge(player_stats, left_on = 'fullName', right_on = 'player', how='right')
df_left[df_left['fullName'].isna() == True][["firstName","middleName","lastName", "fullName","officialName", "player"]]#.head(20)

,firstName,middleName,lastName,fullName,officialName,player
0,NaN,NaN,NaN,NaN,NaN,Christian Eriksen
4,NaN,NaN,NaN,NaN,NaN,Dele Alli
7,NaN,NaN,NaN,NaN,NaN,Son Heung-min
10,NaN,NaN,NaN,NaN,NaN,Mousa Dembélé
17,NaN,NaN,NaN,NaN,NaN,Érik Lamela
...,...,...,...,...,...,...
3159,NaN,NaN,NaN,NaN,NaN,Nacho Vidal
3161,NaN,NaN,NaN,NaN,NaN,Nacho Gil
3162,NaN,NaN,NaN,NaN,NaN,João Cancelo
3165,NaN,NaN,NaN,NaN,NaN,Javier Jiménez García


In [138]:
#df_left.to_csv('player_info_dash.csv')

## Webscraped data graphs

(solo temporales, con mejores métricas usaremos wyscout)

In [133]:
df_events

,Unnamed: 0,eventId,subEventName,playerId,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,...,x_inicio,y_inicio,x_fin,y_fin,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5
0,0,8,Simple pass,25413,2499719,Pass,1609,1H,2.758649,85.0,...,49,49,31.0,78.0,1801.0,NaN,NaN,NaN,NaN,NaN
1,1,8,High pass,370224,2499719,Pass,1609,1H,4.946850,83.0,...,31,78,51.0,75.0,1801.0,NaN,NaN,NaN,NaN,NaN
2,2,8,Head pass,3319,2499719,Pass,1609,1H,6.542188,82.0,...,51,75,35.0,71.0,1801.0,NaN,NaN,NaN,NaN,NaN
3,3,8,Head pass,120339,2499719,Pass,1609,1H,8.143395,82.0,...,35,71,41.0,95.0,1801.0,NaN,NaN,NaN,NaN,NaN
4,4,8,Simple pass,167145,2499719,Pass,1609,1H,10.302366,85.0,...,41,95,72.0,88.0,1801.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628654,628654,8,Simple pass,20623,2565927,Pass,682,2H,2939.077491,85.0,...,66,25,88.0,2.0,1801.0,NaN,NaN,NaN,NaN,NaN
628655,628655,7,Acceleration,122832,2565927,Others on the ball,682,2H,2940.515560,70.0,...,88,2,97.0,21.0,1801.0,NaN,NaN,NaN,NaN,NaN
628656,628656,8,Cross,122832,2565927,Pass,682,2H,2942.098761,80.0,...,97,21,92.0,26.0,401.0,1802.0,NaN,NaN,NaN,NaN
628657,628657,8,Simple pass,40756,2565927,Pass,675,2H,2943.089232,85.0,...,8,74,9.0,56.0,1801.0,NaN,NaN,NaN,NaN,NaN


## VAEP

In [134]:
df_vaep.groupby(by=['first_name', 'last_name', 'team_name', 'player_id', 'type_name']).sum().sort_values(by='vaep_value', ascending=False).head(20)

NameError: name 'df_vaep' is not defined

In [ ]:
df_vaep['type_name'].unique()

In [ ]:
gk_df = df_vaep[df_vaep['type_name'] == 'keeper_save']

In [ ]:
gk_df.groupby(by=['first_name', 'last_name', 'team_name', 'player_id', 'type_name']).sum().sort_values(by='vaep_value', ascending=False).head(20)

In [ ]:
#salvar 

#df_all_info.to_csv('./../../data/wyscout_tabular/players_info_dash.csv')

In [ ]:
#df_events.to_csv('./../../data/wyscout_tabular/all_events.csv')